In [1]:
import photutils
import astropy 
import numpy as np
import matplotlib.pyplot as plt
from photutils.aperture import aperture_photometry
from photutils.aperture import CircularAperture

from astropy.io import fits
import scipy 
from scipy import stats
from scipy.stats import median_abs_deviation



In [12]:
radius = 5.83

measure = ['f090w']
x = []
y = []
xmin=[]
xmax=[]
ymin = []
ymax=[]

for p in measure:
    check_image=fits.open('jwst_glass_combine_dv1_'+p+'_30mas_microJy.fits')
    whole_data = check_image[0].data
    arr = np.zeros((whole_data.shape[1],whole_data.shape[0]))
    x.append(np.arange(0,whole_data.shape[0], 200))    #creating grid across image for cells 200x200 pixels
    y.append(np.arange(0,whole_data.shape[1], 200))
    
    xvals = np.zeros((len(x[0]), 2))
    yvals = np.zeros((len(y[0]), 2))


    k = 200  #starting values 
    l = 200


    while k<=whole_data.shape[0]: #while k is less than x max value of image
        for i in x[0]:           
            xmin.append(i)    #making list [0,200,400...]
            xmax.append(k)    #making list [200,400,600 ...]
            k+=200
    for j in y[0]:
        if l<=whole_data.shape[1]:
            ymin.append(j)
            ymax.append(l)
            l+=200


    pos= open('positions'+ p, 'r')
    xpos = []
    ypos = []
    for line in pos:
        tokens = line.split(',')
        xpos.append(tokens[0])     #positions of apertures with zero flux contained within them on segmentation map
        ypos.append(tokens[1])
    test_arr = np.zeros((len(xpos), 2))
    for i in range(len(xpos)):
        test_arr[i][0]= xpos[i]
        test_arr[i][1] = ypos[i]

    data_arr = np.zeros((len(test_arr),3))  #want to save x,y, flux error value to array

    filename2 = 'local_depth_test'+p
    file = open(filename2, "w")
    for i in range(1):
        for j in range(len(xmax)):
            data = check_image[0].data[xmin[i]:xmax[i],ymin[j]:ymax[j]]   #cutting portion of image to make cell
            xcentroid = (xmax[i]+xmin[i])/2     #defining central coordinates of cell
            ycentroid = (ymax[j]+ymin[j])/2
            for k in range(len(test_arr)):
                if xmin[j]<= test_arr[k][0]<=xmax[j] and  ymin[j]<=test_arr[k][1]<=ymax[j]: 
                    position  = (test_arr[k][0],test_arr[k][1])
                    aperture = CircularAperture(position, r=radius)
                    image = astropy.io.fits.getdata('jwst_glass_combine_dv1_'+p+'_30mas_microJy.fits')
                    aperture_sum = aperture_photometry(image, aperture)

                    data_arr[k][0] = aperture_sum[0][1].value
                    data_arr[k][1] = aperture_sum[0][2].value
                    data_arr[k][2] = aperture_sum[0][3]

            subset = data_arr[:,2][np.where(data_arr[:,2]>=1e-6)] #ensure the histogram does not spike too much at zero

            mad_normal = median_abs_deviation(subset, scale = 'normal')
            file.write(str(xcentroid)+',' + str(ycentroid)+','+ str(mad_normal)+'\n')
            #creates directory for local depth for each cell of image
    file.close()  





